In [ ]:
import numpy as np
from pathlib import Path
import pandas as pd

which_pc = 'local'
if which_pc == 'local':
    habe_path = '/Users/akim/Documents/LCA_files/HABE_2017/'
elif which_pc == 'merlin':
    habe_path = '/data/user/kim_a/LCA_files/HABE_2017'
habe_path = Path(habe_path)
    
filepath11 = habe_path / "HABE091011_Datenbeschreibung_161128UOe.xlsx"
filepath14 = habe_path / "HABE121314_Datenbeschreibung_161128UOe.xlsx"
filepath17 = habe_path / "HABE151617_Datenbeschreibung_191111UOe.xlsx"


In [ ]:
def get_categories(filepath):
    sheet_name = "Ausgaben"
    df_raw = pd.read_excel(filepath, sheet_name=sheet_name, header=10).dropna(how='all')[:-2]
    df = df_raw['Variablenbezeichnung'].map(str) + \
           '-' + df_raw['Unnamed: 1'].map(str) + \
           '-' + df_raw['Unnamed: 2'].map(str) + \
           '-' + df_raw['Unnamed: 3'].map(str) + \
           '-' + df_raw['Unnamed: 4'].map(str)
    return df.values
df11_ausgaben = get_categories(filepath11)
df14_ausgaben = get_categories(filepath14)
df17_ausgaben = get_categories(filepath17)
print(len(df11_ausgaben), len(df14_ausgaben), len(df17_ausgaben), np.all(df14_ausgaben == df17_ausgaben))

# Result:
# Missing category in 2011: 
# 3200.05 Nicht aufteilbare Einkommens- und Vermögenssteuern (Kanton und Gemeinde) [6]
# 3200.05 Non-apportionable income and wealth taxes (canton and commune) [6]

In [ ]:
def get_units(filepath):
    sheet_name = "Mengen"
    df_raw = pd.read_excel(filepath, sheet_name=sheet_name, header=10).dropna(how='all')[:-2]
    df = df_raw['Variablenname'].map(str) + \
           '-' + df_raw['Unnamed: 1'].map(str) + \
           '-' + df_raw['Unnamed: 2'].map(str) + \
           '-' + df_raw['Einheit'].map(str)
    return df.values
df11_units = get_units(filepath11)
df14_units = get_units(filepath14)
df17_units = get_units(filepath17)
print(np.all(df14_units==df17_units), df11_units[df11_units!=df14_units], df14_units[df11_units!=df14_units])

# Result:
# All units are the same

In [ ]:
import brightway2 as bw
bw.projects.set_current("GSA for hybrid")
ei = bw.Database("ecoinvent 3.7.1 cutoff")
ag = bw.Database("Agribalyse 1.3 - ecoinvent 3.7.1 cutoff")
method = ('IPCC 2013', 'climate change', 'GWP 100a')
bw.databases

# Eggs

In [ ]:
# Eggs are not included, but there's poultry
# https://www.ecoinvent.org/support/ecoinvent-forum/topic.html?&tid=1714
chicken_ei = [act for act in ei if "chicken production" in act['name'].lower()][0]
eggs_ag = [act for act in ag if "egg, national average, at farm gate" in act['name'].lower()][0]
lca_chicken = bw.LCA({chicken_ei: 1}, method)
lca_chicken.lci()
lca_chicken.lcia()
score_chicken = lca_chicken.score
lca_eggs = bw.LCA({eggs_ag: 1}, method)
lca_eggs.lci()
lca_eggs.lcia()
score_eggs = lca_eggs.score

print(score_eggs, score_chicken, score_eggs/score_chicken)
# chicken_ei.as_dict()

# Sea bass

In [ ]:
seabass_ag = [act for act in ag if 
              "Sea bass or sea bream, 200-500g, conventional, in cage, at farm gate" == act['name']][0]
lca_seabass = bw.LCA({seabass_ag: 1}, method)
lca_seabass.lci()
lca_seabass.lcia()
score_seabass = lca_seabass.score

marine_fish = [act for act in ei if "market for marine fish" == act['name']][0]
lca_marine_fish = bw.LCA({marine_fish: 1}, method)
lca_marine_fish.lci()
lca_marine_fish.lcia()
score_marine_fish = lca_marine_fish.score

print(score_seabass, score_marine_fish, score_seabass / score_marine_fish)
# marine_fish.as_dict()

# Large trout

In [ ]:
large_trout_ag = [act for act in ag if 
              "Large trout, 2-4kg, conventional, at farm gate" == act['name']][0]
lca_large_trout = bw.LCA({large_trout_ag: 1}, method)
lca_large_trout.lci()
lca_large_trout.lcia()
score_large_trout = lca_large_trout.score

market_for_trout_ei = [act for act in ei if "market for trout" == act['name'].lower()][0]
lca_market_for_trout = bw.LCA({market_for_trout_ei: 1}, method)
lca_market_for_trout.lci()
lca_market_for_trout.lcia()
score_market_for_trout = lca_market_for_trout.score

print(score_large_trout, score_market_for_trout, score_large_trout / score_market_for_trout)
# market_for_trout_ei.as_dict()

# Small trout

In [ ]:
small_trout_ag = [act for act in ag if 
              "Small trout, 250-350g, conventional, at farm gate" == act['name']][0]
lca_small_trout = bw.LCA({small_trout_ag: 1}, method)
lca_small_trout.lci()
lca_small_trout.lcia()
score_small_trout = lca_small_trout.score

print(score_small_trout, score_market_for_trout, score_small_trout / score_market_for_trout)